# ANN Retrieval - Initial Retrieval for Pseudo-Feedback

## Setup and VectorDB Connection

In [1]:
from pinecone import Pinecone
from sentence_transformers import SentenceTransformer
import torch
import pandas as pd

In [2]:
API_KEY = '703c7c8a-2b8f-46bc-b2f7-ede6b037b3fa'
index_name = 'ann-embeddings'
TOP_N=10
TOP_K=100

In [3]:
pc = Pinecone(api_key=API_KEY)
index = pc.Index(index_name)

In [4]:
if torch.cuda.is_available():
    print('using GPU')
    device = 'cuda'
else:
    print('using CPUG')
    device = 'cpu'    

using GPU


In [5]:
model = SentenceTransformer('all-MiniLM-L6-v2')
model = model.to(device)

/home/student/.local/lib/python3.8/site-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


## Query Processing

In [6]:
# query = 'Algorithms for image processing and computer vision'
query = 'Computer vision, the automatic construction of scene descriptions from image input data, has just entered its second decade. Approaches have varied widely, especially in the amounts of symbolic, domain-dependent knowledge and inference that are incorporated into the vision process. Much current research addresses the extraction of physical properties of the scene (depth, surface orientation, reflectance) from images by using only a few general assumptions about the scene domain. Extraction of physical parameters is part of a hierarchy of operations needed to transform image input data to symbolic descriptions. Two other processes that serve as examples are stereo fusion and the partitioning of image phenomena into related groups. Computer vision research is influencing theories of animal perception as well as the design of computing architectures for artificial intelligence.'
embedded_query = model.encode(query, convert_to_tensor=True).cpu().detach().numpy()
embedded_query_list = embedded_query.tolist()


## Top K Retrieval

In [34]:
response = index.query(
    vector=embedded_query_list,
    top_k=TOP_K,
    include_values=False,
    include_metadata=True
)

In [66]:
# data = pd.read_csv('/home/student/FinalProject/PaperFeedback/Datasets/acm_citation_network_v8_labeled.csv')
# data.head(5)
# num_nulls = data['abstract'].isnull().sum()
# print(num_nulls/len(data['abstract']))

In [38]:
print(response)

{'matches': [{'id': '5390a6b120f70186a0e8462d',
              'metadata': {'abstract': 'For human vision to be explained by a '
                                       'computational theory, the first '
                                       'question is plain: What are the '
                                       'problems the brain solves when we see? '
                                       'It is argued that vision is the '
                                       'construction of efficient symbolic '
                                       'descriptions from images of the world. '
                                       'An important aspect of vision is '
                                       'therefore the choice of '
                                       'representations for the different '
                                       'kinds of information in a visual '
                                       'scene. An overall framework is '
                                       'sugges

In [62]:
# print(response['matches'])
matches = response['matches']

matches_with_details=[]
for i,match in enumerate(matches):
    doc_id = match['id']
    score = match['score']
    # title = data[data['id'] == int(doc_id)]['title'].fillna("").values[0]
    #abstract = data[data['index'] == doc_id]['abstract'].fillna("").values[0]
    # text_doc="title- "+title+" abstract- "+abstract
    abstract=match['metadata']['abstract']
    print_check= {
        'id': doc_id,
        'score': score,
        'text': abstract
    }
    print(str(i+1)+'. '+str(print_check)+'\n')

    match_info = {
        'id': doc_id,
        'text': abstract
    }
    matches_with_details.append(match_info)


1. {'id': '5390a6b120f70186a0e8462d', 'score': 0.664827943, 'text': 'For human vision to be explained by a computational theory, the first question is plain: What are the problems the brain solves when we see? It is argued that vision is the construction of efficient symbolic descriptions from images of the world. An important aspect of vision is therefore the choice of representations for the different kinds of information in a visual scene. An overall framework is suggested for extracting shape information from images, in which the analysis proceeds through three representations; (1) the primal sketch, which makes explicit the intensity changes and local two-dimensional geometry of an image, (2) the 2 1/2-D sketch, which is a viewer-centred representation of the depth, orientation and discontinuities of the visible surfaces, and (3) the 3-D model representation, which allows an object-centred description of the threedimensional structure and organization of a viewed shape. The critic

In [65]:
#matches_with_details

## Reranking

In [46]:
reranked_documents = pc.inference.rerank(
    model="bge-reranker-v2-m3",
    query=query,
    documents=matches_with_details,
    top_n=TOP_N,
    return_documents=True
)

In [64]:
reranked_documents

RerankResult(
  model='bge-reranker-v2-m3',
  data=[
    { index=21, score=0.5627172,
      document={id="5390985d20f70186a...", text="Computer vision i..."} },
    { index=58, score=0.55438,
      document={id="5390b0ca20f70186a...", text="The research pres..."} },
    ... (6 more documents) ...,
    { index=74, score=0.22541662,
      document={id="555a1d570cf2b2190...", text="An approach for s..."} },
    { index=54, score=0.21733753,
      document={id="539089d320f70186a...", text="Hardware, softwar..."} }
  ],
  usage={'rerank_units': 1}
)

## Final Retrieval Results

In [63]:
for i,doc in enumerate(reranked_documents.data):
    print(str(i+1)+'. id: '+doc['document']['id']+'   score: '+str(doc['score'])+'   abstract: '+doc['document']['text']+'\n')

1. id: 5390985d20f70186a0e088fb   score: 0.5627172   abstract: Computer vision is the process of using computers to extract from images useful information about the physical world, including meaningful descriptions of physical objects. For example, if an image sensor, such as a digitizing video camera, captured an image of a physical scene, and the digital image was input to a computer vision system, the desired output would be a description of the physical scene in terms that would be useful for the particular task at hand. Computer vision has many applications, including robotics, industrial automation, document processing, remote sensing, navigation, microscopy, medical imaging, and the development of visual prostheses for the blind.

2. id: 5390b0ca20f70186a0ed9506   score: 0.55438   abstract: The research presented in this paper represents several novel conceptual contributions to the computer vision literature. In this position paper, our goal is to define the scope of computer v